In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

from IPython.display import Image as Image_Jupyter

# Les données : Cats vs Dogs (kaggle)

In [ ]:
train_dogs = '../data/cats-dogs/train_resize/dogs/'
train_cats = '../data/cats-dogs/train_resize/cats/'

test = '../data/cats-dogs/test_resize/'

In [ ]:
import PIL
from PIL import Image

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
img = Image.open('../data/cats-dogs/train_resize/cats/10.jpg')
plt.imshow(img)

In [ ]:
ROWS = 100
COLS = 100

# Modèle : AlexNet (2012)

In [ ]:
Image_Jupyter(filename='../ressources/alexnet.png') 

 - Dropout (régularisation)
 - Cropping (augmentation de données)
 - ReLU (fonction d'activation)
 
## Caisse à outils : 

In [ ]:
# Point de départ
from keras.models import Sequential

# Couches nécessaires
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Convolution2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D

# Régularisation ? 
from keras.regularizers import l2

# Algorithme d'optimisation 
from keras.optimizers import SGD, RMSprop

In [ ]:
model = Sequential()

model.add(Convolution2D(96, 11, 11,subsample=(2,2), border_mode='same', input_shape=(ROWS, COLS,3), activation='relu'))
# batch normalization ? 
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2),border_mode='valid'))
model.add(Convolution2D(384, 5, 5,subsample=(1,1), border_mode='same', activation='relu'))
# batch normalization ? 
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2),border_mode='valid'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(384, 3, 3, border_mode='same', activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(384, 3, 3, border_mode='same', activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, 3, 3, border_mode='same', activation='relu'))
model.add(MaxPooling2D((3, 3), strides=(2,2), border_mode='valid'))


model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
    
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1))
model.add(Activation('sigmoid'))

In [ ]:
model.summary()

In [ ]:
#sgd = SGD(lr=0.01)
rms = RMSprop(lr=0.0001)
model.compile(loss='binary_crossentropy', optimizer=rms, metrics=['accuracy'])

# Gérer un gros dataset avec Keras 

- Lorsqu'il est impossible de tout charger en mémoire 

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

- générateur de batch 
- augmentation de données en temps réel



```python
keras.preprocessing.image.ImageDataGenerator(featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=0.,
    width_shift_range=0.,
    height_shift_range=0.,
    shear_range=0.,
    zoom_range=0.,
    channel_shift_range=0.,
    fill_mode='nearest',
    cval=0.,
    horizontal_flip=False,
    vertical_flip=False,
    rescale=None,
    dim_ordering=K.image_dim_ordering())

```

Permet aussi de manipuler des gros dataset :
- une classe par répertoire
- augmentation de données et resize en temps réel 

In [ ]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=10.0)

train_generator = train_datagen.flow_from_directory(
        '../data/cats-dogs/train_resize/',
        target_size=(100, 100),
        batch_size=32,
        class_mode='binary')

In [ ]:
model.fit_generator(
        train_generator,
        samples_per_epoch=25000,
        nb_epoch=50,
        verbose=1)

In [ ]:
!ls

In [ ]:
model.load_weights('model_0.81.h5')

In [ ]:
from quiver_engine import server

In [ ]:
server.launch(model, classes=['cat','dog'], input_folder='../data/cats-dogs/test_resize/', port=7000)